<a href="https://colab.research.google.com/github/tekoryu/rough-on-rats/blob/main/notebooks/model_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Engenharia de Softwares Inteligentes

## TITANIC CHALLENGE

Desafio, dados originais e detalhes da base disponiveis em:

https://www.kaggle.com/competitions/titanic

## Descrição
O conjunto de dados é do tipo categórico, onde precisaremos classificar cada um dos registros da lista como sobrevivente ou não ao famoso naufrágio. Com base em uma lista de treinamento contendo sobreviventes já classificados o objetivo é prever com relativa acurácia em relação aos demais. A estratégia que seguiremos é de _feature engineering_ para obter dados relativos a condição economica, social e de gênero dos passageiros, agrupando ao máximo os casos diversos para depois jogar essas informações como colunas com valores booleanos. A partir daí serão utilizadas as técnicas de particionamento, pipeline e hiperparametrização conforme visto no curso.

In [37]:
import re

import pandas as pd
from sklearn.preprocessing import LabelEncoder


## Carga do Dataset

Os arquivos do dataset possuem um arquivo de teste, que será ignorado e testado apenas ao final.

In [38]:
# Arquivos do Kaggle
url = "https://raw.githubusercontent.com/tekoryu/rough-on-rats/main/data/titanic/train.csv"
url2 = "https://raw.githubusercontent.com/tekoryu/rough-on-rats/main/data/titanic/test.csv"

# Carga no Pandas
df_treino = pd.read_csv(url, delimiter=',')
df_teste = pd.read_csv(url2, delimiter=',')

# Exibe a primeira linha do dataset
df_treino.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Dicionário de Dados

Survival = [0,1] Se a pessoa sobreviveu.
Pclass = [1,2,3] Primeira, Segunda e Terceira Classe
Sex = ["Male", "Female"] Sexo
Age = Idade
SibSP = Irmãos e Conjuges
Parch = Filhos?
Ticket = Parece que tem um cabeçalho, ver depois.
Fare = Valor do ingresso
Cabin = Número da Cabine
Embarked = Inicial do porto de embarque

## Etapa I - Tratamento Inicial dos Dados e Feature Engineering

Da observação dos dados temos como premissa que a definição de quem foi salvo se deu por avaliação de atributos (classe econômica, gênero, idade etc).
Vamos buscar reforçar esses atributos a partir dos dados disponíveis.

### Tratamento de Null e em branco

In [39]:
# Lista a quantidade de campos vazios no dataset
df_treino.isna().sum(axis=0).sort_values(ascending=False)

Cabin          687
Age            177
Embarked         2
PassengerId      0
Name             0
Pclass           0
Survived         0
Sex              0
Parch            0
SibSp            0
Fare             0
Ticket           0
dtype: int64

Como visto, tanto nos arquivos de treino quanto de teste há muitos vazios em Cabin e Age.

In [40]:
# Decide por eliminar a feature 'Cabin' e preencher os 'em branco' da idade com a mediana. O código será reaproveitado para tratar o arquivo de teste.
def preenche_branco(entrada):
    # Deleta a cabine
    entrada.drop('Cabin', axis=1, inplace=True)
    # Preenche as idades ausentes com mediana
    entrada = entrada.fillna(entrada['Age'].median())
    return entrada
    
df_treino = preenche_branco(df_treino)

### Pronomes de Tratamento

Agora é a hora de extrair os pronomes da feature Name e criar uma coluna própria. Depois vamos verificar a possibilidade de criar categorias para esses títulos.

In [41]:
def pega_pronome_tratamento(entrada):
    # A entrada deve ser um dataframe. Pega o Mr. Mrs. e coloca em coluna própria
    pronome_tratamento = re.search(r' ([A-Za-z]+)\.',entrada)
    if pronome_tratamento:
        return pronome_tratamento.group(1)
    return ""

df_treino['Pronome'] = df_treino['Name'].apply(pega_pronome_tratamento)
df_treino.drop('Name', axis=1, inplace=True)

In [42]:
grouped = df_treino.groupby(['Pronome', 'Survived']).size().unstack(fill_value=0)
distribuicao = grouped.div(grouped.sum(axis=1), axis=0) * 100
distribuicao.drop(0, axis=1, inplace=True)
distribuicao

Survived,1
Pronome,
Capt,0.000000
Col,50.000000
Countess,100.000000
Don,0.000000
Dr,42.857143
Jonkheer,0.000000
Lady,100.000000
Major,50.000000
Master,57.500000


Classificarei os passageiros, através do seu pronome pela probabilidade de ser poupado ou de se sacrificar, com base na lista acima. 5 para chances altas de se sacrificar até 1, para chances altas de ser poupado.

In [43]:
estoicismo = {
    'Capt': 5,
    'Col': 3,
    'Countess': 1,
    'Don': 5,
    'Dr': 3,
    'Jonkheer': 5,
    'Lady': 1,
    'Major': 3,
    'Master': 3,
    'Miss': 4,
    'Mlle': 1,
    'Mme': 1,
    'Mr': 2,
    'Mrs': 2,
    'Ms': 1,
    'Rev': 5,
    'Sir': 1
}
df_treino['Pronome'] = df_treino["Pronome"].apply(lambda x: estoicismo.get(x))

In [44]:
df_treino

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Pronome
0,1,0,3,male,22.0,1,0,A/5 21171,7.2500,S,2
1,2,1,1,female,38.0,1,0,PC 17599,71.2833,C,2
2,3,1,3,female,26.0,0,0,STON/O2. 3101282,7.9250,S,4
3,4,1,1,female,35.0,1,0,113803,53.1000,S,2
4,5,0,3,male,35.0,0,0,373450,8.0500,S,2
...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,male,27.0,0,0,211536,13.0000,S,5
887,888,1,1,female,19.0,0,0,112053,30.0000,S,4
888,889,0,3,female,28.0,1,2,W./C. 6607,23.4500,S,4
889,890,1,1,male,26.0,0,0,111369,30.0000,C,2


Em relação ao 'Embarked' iremos substituir os valores de texto por números com auxilio do LabelEncoder. Os dados precisarão ser limpos.

In [47]:
le = LabelEncoder()
df_treino["Embarked"] = df_treino["Embarked"].apply(lambda x: 'A' if x == 28.0 else x ) # Tinha uma linha com float ao inves de integer
df_treino["cod_embarque"] = le.fit_transform(df_treino['Embarked'])
df_treino.drop('Embarked', axis=1, inplace=True)


Vamos repetir o processo para eliminar outros campos de texto.

In [56]:
df_treino["Sex"] = le.fit_transform(df_treino['Sex'])
df_treino.drop(["PassengerId"], axis=1, inplace=True)

Vamos cuidar da questão do ticket, tentando extrair uma lógica de seus códigos.

In [ ]:
def ticket_master:
    # Vamos tentar pegar esses prefixos